# Clean and Combine Scraped Data

A few steps:

- Develop a formatter function (to be saved in the helpers.py file) to reformat raw csvs into a more manageable format.
- Reformat raw csvs and save as one dataframe

In [1]:
import numpy as np
import pandas as pd
import os

from difflib import SequenceMatcher

In [2]:
loc = './Data/Raw Tables/'

fighter_urls = pd.read_csv('.\Data\FighterURLs.csv')
fighter_urls = fighter_urls.set_index('Name')

#We drop these fighters because their URLs were screwy
drop_bad_url = ['André Pederneiras',
                'Christian Morecraft',
                'Colin Fletcher',
                'David Baron',
                'Mark Schultz',
                'Jack Nilson']

# drop some people where we have difficulty with their URLs:
fighter_urls = fighter_urls.drop(drop_bad_url)
fighter_urls.loc['Daniel Kelly','URL'] = 'https://en.wikipedia.org/wiki/Dan_Kelly_(fighter)'
fighter_urls.loc['Maurice Smith','URL'] = 'https://en.wikipedia.org/wiki/Maurice_Smith_(kickboxer)'
fighter_urls.loc['Ryan Hall','URL'] = 'https://en.wikipedia.org/wiki/Ryan_Hall_(grappler)'
fighter_urls.sample(n=20)

,URL
Name,
Rafael dos Anjos,https://en.wikipedia.org/wiki/Rafael_dos_Anjos
Louis Smolka,https://en.wikipedia.org/wiki/Louis_Smolka
Tedd Williams,https://en.wikipedia.org/wiki/Tedd_Williams
Quinton Jackson,https://en.wikipedia.org/wiki/Quinton_Jackson
Shane Carwin,https://en.wikipedia.org/wiki/Shane_Carwin
Phil De Fries,https://en.wikipedia.org/wiki/Phil_De_Fries
Bristol Marunde,https://en.wikipedia.org/wiki/Bristol_Marunde
Joseph Benavidez,https://en.wikipedia.org/wiki/Joseph_Benavidez
Mark Holst,https://en.wikipedia.org/wiki/Mark_Holst


In [3]:
all_raw_csvs = []

for name in fighter_urls.index:
    filename = name.lower().replace(' ','_') +'.pkl'
    df = pd.read_pickle(loc + filename)
    df['Fighter'] = name
    all_raw_csvs.append(df)

raw_data = pd.concat(all_raw_csvs)
raw_data.head()

,,Res.,Record,Opponent,Method,Event,Date,Round,Time,Location,Notes,,birthday,Fighter
0,,Loss,30–14–1,Justin Salas,Decision (split),UFC on Fox: Johnson vs. Moraga,"000000002013-07-27-0000July 27, 2013",3,5:00,"Seattle, Washington, United States",,,1980-12-09,Aaron Riley
1,,Loss,30–13–1,Tony Ferguson,TKO (corner stoppage),UFC 135,"000000002011-09-24-0000September 24, 2011",1,5:00,"Denver, Colorado, United States",,,1980-12-09,Aaron Riley
2,,Win,30–12–1,Joe Brammer,Decision (unanimous),UFC 114,"000000002010-05-29-0000May 29, 2010",3,5:00,"Las Vegas, Nevada, United States",,,1980-12-09,Aaron Riley
3,,Loss,29–12–1,Ross Pearson,TKO (doctor stoppage),UFC 105,"000000002009-11-14-0000November 14, 2009",2,4:34,"Manchester, England",,,1980-12-09,Aaron Riley
4,,Win,29–11–1,Shane Nelson,Decision (unanimous),UFC 101,"000000002009-08-08-0000August 8, 2009",3,5:00,"Philadelphia, Pennsylvania, United States",,,1980-12-09,Aaron Riley


In [4]:
raw_data.shape

(26681, 14)

### Format Date Column

In [5]:
def format_date(date):
    if len(date) > 23:
        return pd.to_datetime(date[8:18],format='%Y-%m-%d')
    else:
        return pd.to_datetime(date)

raw_data['Date_new'] = raw_data['Date'].apply(format_date)

raw_data[['Date','Date_new']].sample(n=5)

,Date,Date_new
15,"000000002007-12-29-0000December 29, 2007",2007-12-29
10,"000000002013-03-16-0000March 16, 2013",2013-03-16
10,"000000002007-07-07-0000July 7, 2007",2007-07-07
2,"000000002011-02-20-0000February 20, 2011",2011-02-20
1,"000000001998-03-13-0000March 13, 1998",1998-03-13


In [6]:
assert np.sum(raw_data.Date_new.isnull()) == 0

### Re-write 'opponent' column

We need to make sure that our list of names matches the opponent names. To check this, we will do a few thing:

    1. Make sure our 'Fighter' column is a subset of our 'Opponent' Column.
    2. For all unmatched opponents, we need to find the most similar fighter name. If we think we have a match, we'll add it to a dictionary that will be responsible for transforming opponent names.
    3. For the 'other' fights, we have to make sure they are actually 'others' and not mispellings..

In [7]:
fighters = raw_data.Fighter.unique()
opponents = raw_data.Opponent.unique()
subset_boolean = np.array([nm in opponents for nm in fighters])

def fighter_subset_perc():
    perc = len(set(raw_data.Opponent).intersection(set(raw_data.Fighter)))/len(set(raw_data.Fighter))
    print('Subset percentage: {}%'.format(str(perc*100)[:5]))
    
fighter_subset_perc()

Subset percentage: 96.99%


In [8]:
unmatched_fighters = fighters[~subset_boolean]
unmatched_fighters

array(['Alexey Oleinik', 'Alptekin Özkiliç', 'Antonio Braga Neto',
       'Damir Hadzovic', 'Dan Spohn', 'Edgar Garcia', 'James Moontasri',
       'Jan Blachowicz', 'Jarred Brooks', 'Jon Olav Einemo', 'João Roque',
       'Jussier da Silva', 'Jörgen Kruth', 'Kevin Ferguson',
       'Lipeng Zhang', 'Marco Polo Reyes', 'Mauricio Rua',
       'Mirko Filipović', 'Mostapha Al-Turk', 'Nathan Quarry',
       'Renee Forte', 'Robert Drysdale', 'Rolles Gracie',
       'Rony Mariano Bezerra', 'Royston Wee', 'Sako Chivitchian',
       "Sean O'Malley", 'Sergio Moraes', 'Teemu Packalen', 'Tiequan Zhang',
       'Yuta Sasaki'], dtype=object)

In [9]:
def common_substring_length(name1, name2):
    return SequenceMatcher(None, name1, name2).find_longest_match(0,len(name1),0,len(name2)).size

unmatched_similarities = {}

for uf in unmatched_fighters:
    print('Working on: {}'.format(uf))
    similiary = pd.Series(np.nan,index=opponents)
    
    for op in similiary.index:
        similiary.loc[op] = common_substring_length(uf, op)
        
    unmatched_similarities[uf] = similiary.sort_values(ascending=False).head(10)

Working on: Alexey Oleinik
Working on: Alptekin Özkiliç
Working on: Antonio Braga Neto
Working on: Damir Hadzovic
Working on: Dan Spohn
Working on: Edgar Garcia
Working on: James Moontasri
Working on: Jan Blachowicz
Working on: Jarred Brooks
Working on: Jon Olav Einemo
Working on: João Roque
Working on: Jussier da Silva
Working on: Jörgen Kruth
Working on: Kevin Ferguson
Working on: Lipeng Zhang
Working on: Marco Polo Reyes
Working on: Mauricio Rua
Working on: Mirko Filipović
Working on: Mostapha Al-Turk
Working on: Nathan Quarry
Working on: Renee Forte
Working on: Robert Drysdale
Working on: Rolles Gracie
Working on: Rony Mariano Bezerra
Working on: Royston Wee
Working on: Sako Chivitchian
Working on: Sean O'Malley
Working on: Sergio Moraes
Working on: Teemu Packalen
Working on: Tiequan Zhang
Working on: Yuta Sasaki


In [10]:
for uf in unmatched_similarities:
    print()
    print(uf)
    print(fighter_urls.loc[uf])
    print('-------------')
    print(unmatched_similarities[uf])


Sergio Moraes
URL    https://en.wikipedia.org/wiki/Sergio_Moraes
Name: Sergio Moraes, dtype: object
-------------
Sérgio Moraes           11.0
Sergio Melo              8.0
Sergio Matias            8.0
Luis Sergio Melo Jr.     8.0
Adriano Moraes           8.0
Sergio Leal              7.0
Sergio Soares            7.0
Daniel Moraes            7.0
Sergio da Silva          7.0
Sergio Gomez             7.0
dtype: float64

Robert Drysdale
URL    https://en.wikipedia.org/wiki/Robert_Drysdale
Name: Robert Drysdale, dtype: object
-------------
Robert Lalonde      7.0
Robert Wynne        7.0
Robert Whittaker    7.0
Robert Hogan        7.0
Robert Neal         7.0
Robert Thompson     7.0
Robert Irizarry     7.0
Robert Fabrizi      7.0
Robert Lieske       7.0
Robert Balicki      7.0
dtype: float64

João Roque
URL    https://en.wikipedia.org/wiki/João_Roque
Name: João Roque, dtype: object
-------------
RoqueJoão Roque         10.0
Roque Oliver             5.0
João Paulo Rodrigues     5.0
João Paulo 

In [11]:
# We drop these fighters because they are never opponents in the mma fight record tables. This means
# they almost wholely fight fighters outside our dataset.

drop_outsider =['Alexey Oleinik',
                'Sako Chivitchian',
                'Royston Wee',
                'Jörgen Kruth',
                'James Moontasri',
                'Teemu Nurkkala',
                'Robert Drysdale',
                'Jarred Brooks',
                "Sean O'Malley",
                'Teemu Packalen']

raw_data = raw_data.loc[~raw_data['Fighter'].isin(drop_outsider),:]

In [12]:
opponent_name_mapper = {}
opponent_name_mapper['Zhang Lipeng'] = 'Lipeng Zhang'
opponent_name_mapper['Jussier Formiga'] = 'Jussier da Silva'
opponent_name_mapper['Ulka Sasaki'] = 'Yuta Sasaki'
opponent_name_mapper['Mirko Cro Cop'] = 'Mirko Filipović'
opponent_name_mapper['Antônio Braga Neto'] = 'Antonio Braga Neto'
opponent_name_mapper['Polo Reyes'] = 'Marco Polo Reyes'
opponent_name_mapper['Alp Ozkilic'] = 'Alptekin Özkiliç'
opponent_name_mapper['Nate Quarry'] = 'Nathan Quarry'
opponent_name_mapper['Rolles Gracie Jr.'] = 'Rolles Gracie'
opponent_name_mapper['Damir Hadžović'] = 'Damir Hadzovic'
opponent_name_mapper['Jan Błachowicz'] = 'Jan Blachowicz'
opponent_name_mapper['Daniel Spohn'] = 'Dan Spohn'
opponent_name_mapper['Sérgio Moraes'] = 'Sergio Moraes'
opponent_name_mapper['Renée Forte'] = 'Renee Forte'
opponent_name_mapper['Edgar García'] = 'Edgar Garcia'
opponent_name_mapper['Mostapha al-Turk'] = 'Mostapha Al-Turk'
opponent_name_mapper['RoqueJoão Roque'] = 'João Roque'
opponent_name_mapper['John-Olav Einemo'] = 'Jon Olav Einemo'
opponent_name_mapper['Zhang Tiequan'] = 'Tiequan Zhang'
opponent_name_mapper['Rony Jason'] = 'Rony Mariano Bezerra'
opponent_name_mapper['Maurício Rua'] = 'Mauricio Rua'
opponent_name_mapper['Maurício Rua'] = 'Mauricio Rua'
opponent_name_mapper['Kimbo Slice'] = 'Kevin Ferguson'

raw_data['Opponent'] = raw_data['Opponent'].apply(lambda x: opponent_name_mapper[x] if x in opponent_name_mapper else x) 

In [13]:
fighter_subset_perc()

Subset percentage: 100.0%


### Now we need to figure out which opponents that aren't on our list are actually just misspellings..

In [14]:
def perc_opponent_recognized():
    perc = np.mean([nm in fighters for nm in raw_data.Opponent])
    print('Percent opponents are familiar fighters: {}%'.format(str(perc*100)[:5]))
    
perc_opponent_recognized()

Percent opponents are familiar fighters: 38.36%


Hm, OK, let's go by the most frequent opponents seen.

In [15]:
unrecognized = set(opponents) - set(fighters)
unrecognized_counts = raw_data.loc[raw_data.Opponent.isin(unrecognized),'Opponent'].value_counts()
unrecognized_counts.head(15)

Osami Shibuya        33
Ikuhisa Minowa       31
Kiuma Kunioku        30
Shannon Ritch        30
Fedor Emelianenko    27
Ryushi Yanagisawa    27
Masakatsu Funaki     25
Minoru Suzuki        24
Dennis Reed          23
Igor Vovchanchyn     20
Kazuo Misaki         20
Takafumi Ito         19
Shinya Aoki          18
Victor Moreno        18
Derrick Noble        17
Name: Opponent, dtype: int64

OK, let's consider anyone with more than 10 fights. We'll look at their similarities with fighters we have and see if we find anything.

In [16]:
opponents_over10 = unrecognized_counts.index[unrecognized_counts>15]
unmatched_similarities_opp = {}

for op in opponents_over10:
    print('Working on: {}'.format(op))
    similiary = pd.Series(np.nan,index=fighters)
    
    for fg in similiary.index:
        similiary.loc[fg] = common_substring_length(fg, op)
        
    unmatched_similarities_opp[op] = similiary.sort_values(ascending=False).head(10)

Working on: Osami Shibuya
Working on: Ikuhisa Minowa
Working on: Kiuma Kunioku
Working on: Shannon Ritch
Working on: Fedor Emelianenko
Working on: Ryushi Yanagisawa
Working on: Masakatsu Funaki
Working on: Minoru Suzuki
Working on: Dennis Reed
Working on: Igor Vovchanchyn
Working on: Kazuo Misaki
Working on: Takafumi Ito
Working on: Shinya Aoki
Working on: Victor Moreno
Working on: Derrick Noble
Working on: Ruben Villareal
Working on: Kiyoshi Tamura
Working on: Satoru Kitaoka
Working on: Manabu Yamada
Working on: William Hill
Working on: Hiromitsu Kanehara
Working on: Thomas Denny
Working on: Tony Lopez
Working on: Paul Jenkins


In [17]:
for op in unmatched_similarities_opp:
    print()
    print(op)
    print('-------------')
    print(unmatched_similarities_opp[op])


Kiyoshi Tamura
-------------
Issei Tamura             8.0
Kuniyoshi Hironaka       7.0
Tsuyoshi Kohsaka         6.0
Kazuhiro Nakamura        5.0
Satoshi Honma            5.0
Keita Nakamura           5.0
Michinori Tanaka         4.0
Yoshiyuki Yoshida        4.0
Dustin Kimura            4.0
Gadzhimurad Antigulov    4.0
dtype: float64

Osami Shibuya
-------------
Jake Shields          4.0
Kamal Shalorus        3.0
Wanderlei Silva       3.0
Jimi Manuwa           3.0
Ken Shamrock          3.0
Genki Sudo            3.0
Keiichiro Yamamiya    3.0
Kazushi Sakuraba      3.0
Andrei Semenov        3.0
Anthony Hamilton      3.0
dtype: float64

Dennis Reed
-------------
Dennis Siver        7.0
Dennis Hallman      7.0
Dennis Bermudez     7.0
Luis Ramos          4.0
Yosdenis Cedeno     4.0
Denis Kang          4.0
Chris Kelades       3.0
Chris Leben         3.0
Chris Holdsworth    3.0
Chris Haseman       3.0
dtype: float64

Hiromitsu Kanehara
-------------
Masanori Kanehara     9.0
Kuniyoshi Hironaka 

In [18]:
fighter_urls.loc['Masanori Kanehara']

URL    https://en.wikipedia.org/wiki/Masanori_Kanehara
Name: Masanori Kanehara, dtype: object

In [19]:
opponent_name_mapper2 = {}
opponent_name_mapper2['Tank Abbott'] = 'David Abbott'
opponent_name_mapper2['Ovince Saint Preux'] = 'Ovince St. Preux'
opponent_name_mapper2['Oleksiy Oliynyk'] = 'Alexey Oleinik'

raw_data['Opponent'] = raw_data['Opponent'].apply(lambda x: opponent_name_mapper2[x] if x in opponent_name_mapper2 else x) 
perc_opponent_recognized()

Percent opponents are familiar fighters: 38.52%


### Final reformat and save




In [21]:
raw_data['birthday'] = pd.to_datetime(raw_data['birthday'], errors='coerce')

Index(['', 'Res.', 'Record', 'Opponent', 'Method', 'Event', 'Date', 'Round',
       'Time', 'Location', 'Notes', '', 'birthday', 'Fighter', 'Date_new'],
      dtype='object')

In [37]:
raw_data = raw_data.loc[~raw_data['Res.'].isin(['TBD','NC','']),:]
raw_data['Res.'].unique()

array(['Loss', 'Win', 'Draw', 'win', 'loss', 'Won', 'Winx', 'Lossx', 'xWin'], dtype=object)

In [41]:
Res_map = {}
Res_map['Loss'] = -1
Res_map['Lossx'] = -1
Res_map['loss'] = -1
Res_map['Win'] = 1
Res_map['win'] = 1
Res_map['Won'] = 1
Res_map['Winx'] = 1
Res_map['xWin'] = 1
Res_map['Draw'] = 0

raw_data['Result'] = raw_data['Res.'].apply(lambda x: Res_map[x])

In [43]:
raw_data.head(3)

,,Res.,Record,Opponent,Method,Event,Date,Round,Time,Location,Notes,,birthday,Fighter,Date_new,Result
0,,Loss,30–14–1,Justin Salas,Decision (split),UFC on Fox: Johnson vs. Moraga,"000000002013-07-27-0000July 27, 2013",3,5:00,"Seattle, Washington, United States",,,1980-12-09,Aaron Riley,2013-07-27,-1
1,,Loss,30–13–1,Tony Ferguson,TKO (corner stoppage),UFC 135,"000000002011-09-24-0000September 24, 2011",1,5:00,"Denver, Colorado, United States",,,1980-12-09,Aaron Riley,2011-09-24,-1
2,,Win,30–12–1,Joe Brammer,Decision (unanimous),UFC 114,"000000002010-05-29-0000May 29, 2010",3,5:00,"Las Vegas, Nevada, United States",,,1980-12-09,Aaron Riley,2010-05-29,1


In [47]:
final_columns = ['Fighter','Opponent','Date_new','birthday','Result','Round','Time','Location','Method']

data = raw_data[final_columns]
data = data.rename(columns = {'birthday':'fighter_birthday','Date_new':'Date'})
data.head(3)
data.to_csv('./Data/fights.csv',index=False)